# Import Libraries

In [1]:
import numpy as np
import statistics
import math

import pickle
import seaborn as sns

import pandas as pd
from pandas import DataFrame

import scipy
from scipy import signal, stats
from scipy.fft import fft, ifft
from scipy.signal import find_peaks, butter, filtfilt, iirnotch, savgol_filter, welch, lfilter, freqz, argrelextrema
from scipy.stats import iqr
from scipy.integrate import simps
from scipy.interpolate import UnivariateSpline

import cvxopt as cv
import cvxopt.solvers

# Load Data

In [2]:
def load_data() :
    """Extract all the data from the pickle files in the folder

    Parameters
    ----------
    Return
    ------
    data : 1d-array
      list of dictionaries
    """  
        
    data=[]
    for i in range (1,45):
        if i < 10 :
            div='0' 
        else : 
            div=''
        with open("data/participant_"+div+str(i)+".pkl" ,"rb") as f : 
            data.append(pickle.load(f))
    return data

data=load_data()

# Feature extraction and data analysis

## Global Helper Functions

In [3]:
def percentQ1Q3(x, mean, std):
    """Return the percentage of times the value is above μ + σ and below μ − σ of the corresponding feature.

    Parameters
    ----------
    x : 1d-array
      Input feature
    mean : float
      Mean
    std : float
      Standard deviation

    Return
    ------
    return 1 : float
      Percentage of times the value is below μ − σ
    return 2 : float
      Percentage of times the value is above μ + σ
    """ 
    
    if len(x) == 0 :
        
        return 0,0
    
    idx_1 = [number for number in x if number < mean-std]

    idx_3 = [number for number in x if number > mean+std] 
    
    return round(len(idx_1)/len(x),3), round(len(idx_3)/len(x),3)

####################################################################################################################

def last50Sec(time, data):
    
    """Return the last 50 seconds of the signal

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    time : 1d-array
      Time in milliseconds

    Return
    ------
    data50 : 1d-array
      Last 50 seconds of the input signal
    time50 : 1d-array
      Last 50 seconds of the input time
    idx : int
      First valid index in the original data trace
    """    
    data50, time50, i = [], [], 1
    
    end = time[len(time)-i] - 50*1000
    
    while(time[len(time)-i] > end):
        
        idx=len(time)-i
        
        data50.append(data[idx])
        time50.append(time[idx])
        
        i=i+1
        
    data50 = data50[::-1]
    time50 = time50[::-1]
    
    
    return data50, time50, idx

####################################################################################################################

def bandpower(data, sf, band, nfft=None, additional=False):
    """Compute the average, min/max, std of power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
      
    Return
    ------
    bp : float
      Absolute band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps

    band = np.asarray(band)
    low, high = band
    freqs, psd = welch(data, sf, nperseg=sf*15, noverlap=sf*10, nfft=nfft)

    
    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    
    # Average - Integral approximation of the spectrum using parabola (Simpson's rule) 
    freq_res = freqs[1] - freqs[0]
    avg = round(simps(psd[idx_band], dx=freq_res),3)
        
    if additional == True : 
        
        # Min/Max for additional feat
        maxi=max(psd[idx_band])
        mini=min(psd[idx_band])
    
        # Std
        std=statistics.stdev(psd[idx_band])
        
        return maxi, mini, std, avg
        
    return avg    

## ECG Data

### ECG Helper Functions

In [4]:
def loadL1(data, clip):
    """Load lead 1 from the data

    Parameters
    ----------
    data : dict
      dictionary of the participant.
    clip : int
      number of clip
      
    Return
    ------
    time : 1d-array
      The time series for the given clip when data is available.
    l1 : 1d-array
      The lead 1 series for the given clip when data is available.
    fs : float
      Sampling frequency.      
    mean_negatives : float
      Trimmed mean of data in the forth Cartesian quadrant.
    """
    
    nECG=data["recordings"][clip]['ECG']
    fs=data["FS_ECG"]
    
    l1, time, negatives = [], [],[]
    
    for i in range (0,len(nECG)):
        
        if math.isnan(nECG[i][2]) or math.isnan(nECG[i][1]) or  math.isnan(nECG[i][0]) :
            continue
            
        else :
            lead1=nECG[i][2] - nECG[i][1]
            l1.append(lead1)
            time.append(nECG[i][0])
            if lead1<0 : 
                negatives.append(lead1)
    
    numneg= len(negatives)
    numtot= len(l1)
    ratio = numneg/numtot
    
    mean_negatives=stats.trim_mean(negatives, 0.25)
    
    return time, l1, fs, mean_negatives

####################################################################################################################

def paper_lowpass(data, mean_negatives):
    """Low pass filter using the scipy library

    Parameters
    ----------
    data : 1d-array
      Input signal
    cutoff : float
      Cutoff frequency
    fs : float
      Sampling frequency
    order : int
      Order of the filter

    Return
    ------
    y : 1d-array
      Filtered signal
    """  
    
    
    noise=[0]*len(data)
    
    noise[0]=data[0]
    
    alpha = 1/8
    
    for i in range (1, len(data)) : 
        
        try:
            
            noise[i]=noise[i-1]+alpha*data[i]-alpha*data[i-8]
        
        except IndexError:
            
            noise[i]=data[i]
            
    if stats.trim_mean(data, 0.25)<0 : 
        
        filtered_data=np.array(data)-np.array(noise)-[mean_negatives]*len(data)
        
    else : 
        
        filtered_data=np.array(data)-np.array(noise)
    
    return filtered_data



####################################################################################################################

def ArtifactsDetection(data, time):
    """Detect artifacts in the ECG data (alg from the lecture)

    Parameters
    ----------
    data : 1-d array
      lead 1
    time : 1-d array
      time
      
    Return
    ------
    peaks : 1d-array
      Detected peaks
    peaks Valid : 1d-array
      Valid peaks
    """
    peaks, _ = find_peaks(data, distance =150, prominence=90, width=3)
    
    if len(peaks)==0 : 
        return [0,1,2],[0,0,0]
    
    beats=[0]*len(peaks)
    
    for i in range (1, len(peaks)) : 
        beats[i]=peaks[i]-peaks[i-1]
        
    MedianBeat=statistics.median(beats)
    QD=iqr(beats)/2
    MAD=(MedianBeat-2.9*QD)/3
    MED=3.32*QD
    CBD=(MAD+MED)/2

    peaksValid=[0]*len(peaks)
    Rnn_last=time[peaks[len(peaks)-2]]- time[peaks[len(peaks)-1]]

    for i in range (2, len(peaks)-1) :   

        Rnn=time[peaks[i]]- time[peaks[i-1]]#current
        Rnn_minus1=time[peaks[i-1]]- time[peaks[i-2]]#previous
        Rnn_plus1=time[peaks[i]]- time[peaks[i+1]]#successor
        
        if Rnn_minus1<=2000 and Rnn_minus1>=300 :
            
            if abs(Rnn-Rnn_minus1)<=CBD :
                
                peaksValid[i]=1
                peaksValid[i-1]=1
        else :
            
            if abs(Rnn-Rnn_last)<=CBD :
                
                peaksValid[i]=1
                peaksValid[i-1]=1
                
            else :
                
                if abs(Rnn-Rnn_minus1)<=CBD and abs(Rnn-Rnn_plus1)<=CBD :
                    
                    peaksValid[i]=1
                    peaksValid[i-1]=1
                    
    return peaks, peaksValid

### ECG Class

In [5]:
import matplotlib.pyplot as plt
%matplotlib widget

class ECG:      
    
    def __init__(self, number, clip):
        
        participant_data = data[number-1]
        
        self.time, self.l1, self.fs, mean_negatives = loadL1(participant_data, clip)

        self.l1_filter = paper_lowpass(self.l1, mean_negatives)
        
        self.peaks, self.peaksValid = ArtifactsDetection(self.l1_filter, self.time)
        
        self.f, self.Pxx_den = signal.welch(self.l1_filter, self.fs, nperseg=self.fs*15, noverlap=self.fs*10, nfft=len(self.l1_filter))
        
        self.time50, self.l50, self.idx = last50Sec(self.time, self.l1_filter)

    #####################################################################################################        
     
    def plot_PSD(self, band):
        
        f2, Pxx_den2 = signal.welch(self.l1, self.fs, nperseg=self.fs*15, noverlap=self.fs*10)
        minFrequencies, maxFrequencies = band
        plt.figure(figsize=(10,5))
        plt.semilogy(self.f, self.Pxx_den, label="Filtered")
        plt.semilogy(f2, Pxx_den2, label="Raw")
        plt.xlim([minFrequencies, maxFrequencies])
        plt.legend()
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('PSD')
        plt.show()
        
    #####################################################################################################        
     
    def plot_ECG(self, band = [300000, 310000]):
        
        #minTime, maxTime = band

        plt.figure(figsize=(10,5))
        plt.plot(self.time, self.l1_filter, label ='L1 with low pass')
        #plt.plot(self.time, self.l1, label ='Lead 1 - as is')
        #plt.xlim([minTime, maxTime])
        plt.legend()
        plt.xlabel('Time [ms]')
        plt.ylabel('')
        plt.show()
        
    #####################################################################################################        
     
    def invalidData(self):
        
        ##Percentage of invalid ECG
        count=0
        for i in range (1, len(self.peaksValid)-1) :
            
            if self.peaksValid[i]==0 :    
                start_invalid=int((self.peaks[i-1]+self.peaks[i])/2)
                end_invalid=int((self.peaks[i]+self.peaks[i+1])/2)
                count=count+len(self.l1_filter[start_invalid:end_invalid])

        invalid_data=round(count/len(self.l1_filter)*100,2)
        print("Percentage of invalid data : ",invalid_data, "%")
        
    #####################################################################################################        
     
    def plot_Peaks(self):
        
        posValid=[i for i, e in enumerate(self.peaksValid) if e == 1]
        
        if len(posValid)==0 :
            plt.plot(self.l1_filter)
            plt.show()  
            return 
        
        normal_peaks=self.peaks[posValid]
        plt.figure(figsize=(10,5))
        plt.plot(self.l1_filter)
        plt.plot(self.peaks, self.l1_filter[self.peaks], "x")
        plt.plot(normal_peaks, self.l1_filter[normal_peaks], "o")
        plt.show()  
    
    #####################################################################################################    

    def ECG_stats(self):
        IBI, HR, HRV, tmpHRV = [], [], [], []
        windowHRV=0
        
        peaks50 = [item for item in self.peaks if item > self.idx]
        peaksValid50=self.peaksValid[len(self.peaks)-len(peaks50):]
        
        if sum(peaksValid50) == 0 : 
            
            dict= {
            "ECG_vs1":"nan",
            "ECG_vs2":"nan",
            "ECG_vs3":"nan",
            "ECG_vs4":"nan",
                                         
            "ECG_vs_max":"nan",
            "ECG_vs_min":"nan",
            "ECG_vs_area":"nan",
            "ECG_vs_std":"nan",
            
            "ECG_s1":"nan",
            "ECG_s2":"nan",
            "ECG_s3":"nan",
            "ECG_s4":"nan",
            "ECG_s5":"nan",
            "ECG_s6":"nan",
            "ECG_s7":"nan",
            "ECG_s8":"nan",
            "ECG_s9":"nan",
            "ECG_s10":"nan",
            
            "ECG_h1":"nan",
            "ECG_h2":"nan",
            "ECG_h3":"nan",
            "ECG_h4":"nan",
            "ECG_h5":"nan",
            "ECG_h6":"nan",
            "ECG_h7":"nan",
            "ECG_h8":"nan",
            "ECG_h9":"nan",
            "ECG_h10":"nan",
            
            "ECG_IBI_mean":"nan", 
            "ECG_IBI_std":"nan",
            "ECG_IBI_skw":"nan", 
            "ECG_IBI_krt":"nan",  
            "ECG_IBI_PERQ1":"nan",
            "ECG_IBI_PERQ3":"nan",
            "ECG_IBI_MIN":"nan",
            "ECG_IBI_MAX":"nan",
            
            "ECG_HR_mean":"nan", 
            "ECG_HR_std":"nan",
            "ECG_HR_skw":"nan", 
            "ECG_HR_krt":"nan",  
            "ECG_HR_PERQ1":"nan",
            "ECG_HR_PERQ3":"nan",
            "ECG_HR_MIN":"nan",
            "ECG_HR_MAX":"nan",
            
            "ECG_HRV_mean":"nan", 
            "ECG_HRV_std":"nan",
            "ECG_HRV_skw":"nan", 
            "ECG_HRV_krt":"nan",  
            "ECG_HRV_PERQ1":"nan",
            "ECG_HRV_PERQ3":"nan",
            "ECG_HRV_MIN":"nan",
            "ECG_HRV_MAX":"nan",
        }
            return dict
        
        for i in range (1, len(peaks50)-1) :
            
            if peaksValid50[i]==1 and peaksValid50[i-1]==1:
                
                interval=self.time[peaks50[i]]- self.time[peaks50[i-1]]
                IBI.append(interval)
                HR.append(60000/interval) #Standard for regular heart beats
                
                tmpHRV.append(interval)
                windowHRV=windowHRV+tmpHRV[len(tmpHRV)-1]
                
            if (peaksValid50[i]==0) : #correction due to non-consecutives peaks
                
                windowHRV=0
                tmpHRV = []
                
            if (windowHRV>5000) :
                
                RMSSD=0
                
                for j in range(0,len(tmpHRV)-2) :
                    RMSSD=RMSSD+pow(tmpHRV[j]-tmpHRV[j+1],2)
                
                HRV.append(math.sqrt(RMSSD/(len(tmpHRV)-2)))
                windowHRV=windowHRV-tmpHRV[0]
                tmpHRV.pop(0)
        
        IBI_stats=stats.describe(IBI) 
        IBI_mean, IBI_std = IBI_stats[2], math.sqrt(IBI_stats[3])        
        
        HR_stats=stats.describe(HR) 
        HR_mean, HR_std = HR_stats[2], math.sqrt(HR_stats[3])
        
        if len(HRV) == 0 : 
            HRV_stats=["nan",["nan","nan"],"nan","nan","nan","nan"]
            HRV_mean, HRV_std = "nan","nan"
            HRV_PERQ1, HRV_PERQ3 ="nan","nan"
        else :
            HRV_stats=stats.describe(HRV) 
            HRV_mean, HRV_std = HRV_stats[2], math.sqrt(HRV_stats[3])
            HRV_PERQ1, HRV_PERQ3 = percentQ1Q3(HRV, HRV_mean, HRV_std)
        
        IBI_PERQ1, IBI_PERQ3 = percentQ1Q3(IBI, IBI_mean, IBI_std)
        HR_PERQ1, HR_PERQ3 = percentQ1Q3(HR, HR_mean, HR_std)
        
        # 4 very slow responses
        num=4
        band = [0,0.04]
        vs=[]
        subSTART=band[0]
        for i in range (0,4) : 
            subEND=subSTART+0.025
            vs.append(bandpower(np.array(self.l50), self.fs, [subSTART,subEND], nfft=len((self.l50))))
            subSTART=subSTART+0.005
        
        
        # 10 slow responses
        num=10
        band = [0,2.4]
        s= []
        subSTART=band[0]
        for i in range (0,10) : 
            subEND=subSTART+band[1]/num
            s.append(bandpower(np.array(self.l50), self.fs, [subSTART,subEND]))
            subSTART=subEND
        
        
        # 10 high responses
        num=10
        band = [0,100]
        h= []
        subSTART=band[0]
        for i in range (0,10) : 
            subEND=subSTART+band[1]/num
            h.append(bandpower(np.array(self.l50), self.fs, [subSTART,subEND]))
            subSTART=subEND

        vs_maxi, vs_mini, vs_std, vs_area = bandpower(np.array(self.l50), self.fs, [0,0.04], nfft=len(self.l50) , additional= True)

        ECG_dict = {
            "ECG_vs1":vs[0],
            "ECG_vs2":vs[1],
            "ECG_vs3":vs[2],
            "ECG_vs4":vs[3],
                                         
            "ECG_vs_max":vs_maxi,
            "ECG_vs_min":vs_mini,
            "ECG_vs_area":vs_area,
            "ECG_vs_std":vs_std,
            
            "ECG_s1":s[0],
            "ECG_s2":s[1],
            "ECG_s3":s[2],
            "ECG_s4":s[3],
            "ECG_s5":s[4],
            "ECG_s6":s[5],
            "ECG_s7":s[6],
            "ECG_s8":s[7],
            "ECG_s9":s[8],
            "ECG_s10":s[9],
            
            "ECG_h1":h[0],
            "ECG_h2":h[1],
            "ECG_h3":h[2],
            "ECG_h4":h[3],
            "ECG_h5":h[4],
            "ECG_h6":h[5],
            "ECG_h7":h[6],
            "ECG_h8":h[7],
            "ECG_h9":h[8],
            "ECG_h10":h[9],
            
            "ECG_IBI_mean":round(IBI_mean,3), 
            "ECG_IBI_std":round(IBI_std,3),
            "ECG_IBI_skw":round(IBI_stats[4],3), 
            "ECG_IBI_krt":round(IBI_stats[5],3),  
            "ECG_IBI_PERQ1":round(IBI_PERQ1,3),
            "ECG_IBI_PERQ3":round(IBI_PERQ3,3),
            "ECG_IBI_MIN":round(IBI_stats[1][0],3),
            "ECG_IBI_MAX":round(IBI_stats[1][1],3),
            
            "ECG_HR_mean":round(HR_mean,3), 
            "ECG_HR_std":round(HR_std,3),
            "ECG_HR_skw":round(HR_stats[4],3), 
            "ECG_HR_krt":round(HR_stats[5],3),  
            "ECG_HR_PERQ1":round(HR_PERQ1,3),
            "ECG_HR_PERQ3":round(HR_PERQ3,3),
            "ECG_HR_MIN":round(HR_stats[1][0],3),
            "ECG_HR_MAX":round(HR_stats[1][1],3),
            
            "ECG_HRV_mean":HRV_mean, 
            "ECG_HRV_std":HRV_std,
            "ECG_HRV_skw":HRV_stats[4], 
            "ECG_HRV_krt":HRV_stats[5],  
            "ECG_HRV_PERQ1":HRV_PERQ1,
            "ECG_HRV_PERQ3":HRV_PERQ3,
            "ECG_HRV_MIN":HRV_stats[1][0],
            "ECG_HRV_MAX":HRV_stats[1][1],
        }
        
        
        return ECG_dict

### Answer to questions

#### 1

Plot the Power Spectral Density (PSD) for the Lead I ECG signal of the first participant watching Clip 1 from 0–40 Hz

In [6]:
ECG_1_1=ECG(1,1)

In [7]:
import matplotlib.pyplot as plt
%matplotlib widget

ECG_1_1.plot_PSD(band=[0,40])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 2

*Propose and reason about, implement and apply a suitable low-pass filter to remove noise. Plot the filter’s frequency response. Plot the filtered ECG signal in the time-domain.*

According to [2], common frequencies of the important components on the ECG:

- Heart rate: 0.67 – 5 Hz (i.e. 40 – 300 bpm)
- P-wave: 0.67 – 5 Hz
- QRS: 10 – 50 Hz
- T-wave: 1 – 7 Hz
- High frequency potentials: 100 – 500 Hz

The common frequencies of the artifact and noise on the ECG:

- Muscle: 5 – 50 Hz
- Respiratory: 0.12 – 0.5 Hz (e.g. 8 – 30 bpm)
- External electrical: 50 or 60 Hz (A/C mains or line frequency)
- Other electrical: typically >10 Hz (muscle stimulators, strong magnetic fields, pacemakers with impedance monitoring)

The skin-electrode interface requires special note, as it is the largest source of interference, producing a DC component of 200-300 mV. Compare this to the electrical activity of your heart, which is in the range of 0.1 to 2 mV! The interference seen from this component is magnified by motion, either patient movement, or respiratory variation.

Low-pass filters on the ECG are used to remove high frequency muscle artifact and external interference. They typically attenuate only the amplitude of higher frequency ECG components. Analog low-pass filtering has a noticeable affect on the QRS complex, epsilon, and J-waves but do not alter repolarization signals.


In [8]:
import matplotlib.pyplot as plt
%matplotlib widget

ECG_1_1.plot_ECG()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 3

*Using the algorithm for artifact detection introduced in Lecture 6 or another suitable approach, compute the percentage of ECG data that is flagged as having artifacts. Select one such ECG signal that contains artifacts and plot it.*

In [9]:
ECG_1_1.invalidData()

Percentage of invalid data :  3.31 %


In [10]:
import matplotlib.pyplot as plt
%matplotlib widget


ECG_1_1.plot_Peaks()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 4 

*For the last 50 seconds of each recording, extract the following features as mentioned in the paper
#01 ten low frequency ([0–2.4] Hz) PSDs
#02 four very slow response ([0–0.04] Hz) PSDs
#03 statistical measurements over inter beat intervals
#04 statistical measurements over heart rate
#05 statistical measurements over heart rate variability *


In [11]:
### Attention used to collect features names
features=[list(ECG_1_1.ECG_stats().keys())]

In [12]:
ECG_1_1.ECG_stats()

{'ECG_vs1': 155450.379,
 'ECG_vs2': 0.0,
 'ECG_vs3': 0.0,
 'ECG_vs4': 623053.455,
 'ECG_vs_max': 46760345.749918,
 'ECG_vs_min': 38.146972656835324,
 'ECG_vs_area': 726269.219,
 'ECG_vs_std': 23813756.613865122,
 'ECG_s1': 4864870.465,
 'ECG_s2': 1665.935,
 'ECG_s3': 32.101,
 'ECG_s4': 6.764,
 'ECG_s5': 1.212,
 'ECG_s6': 0.436,
 'ECG_s7': 0.14,
 'ECG_s8': 0.04,
 'ECG_s9': 0.029,
 'ECG_s10': 0.014,
 'ECG_h1': 5860006.459,
 'ECG_h2': 0.0,
 'ECG_h3': 0.0,
 'ECG_h4': 0.0,
 'ECG_h5': 0.0,
 'ECG_h6': 0.0,
 'ECG_h7': 0.0,
 'ECG_h8': 0.0,
 'ECG_h9': 0.0,
 'ECG_h10': 0.0,
 'ECG_IBI_mean': 891.001,
 'ECG_IBI_std': 31.348,
 'ECG_IBI_skw': 0.029,
 'ECG_IBI_krt': -0.197,
 'ECG_IBI_PERQ1': 0.173,
 'ECG_IBI_PERQ3': 0.154,
 'ECG_IBI_MIN': 812.5,
 'ECG_IBI_MAX': 957.031,
 'ECG_HR_mean': 67.422,
 'ECG_HR_std': 2.378,
 'ECG_HR_skw': 0.164,
 'ECG_HR_krt': -0.064,
 'ECG_HR_PERQ1': 0.154,
 'ECG_HR_PERQ3': 0.173,
 'ECG_HR_MIN': 62.694,
 'ECG_HR_MAX': 73.846,
 'ECG_HRV_mean': 30.65323650175001,
 'ECG_HRV_std'

#### 5 

Statistical measurements (band power area, standard deviation, min and max) of the entire 0-0.04Hz band were also collected as additional features, because the integrated power of the four sub-bands in the same frequency domain was every so often unresponsive. The PSD frequencies band 0-100Hz was divided in 10 sub bands and their band power was integrated to collect additional features. These features were said to contain information about emotion recognition in a previous work (see report). 
As a personal curiosity, the maximum and minimum measurement in HR, HRV and IBI was extracted to study to what extent the greatest/smallest HR/HRV/IBI can have a role to predict emotional state.



## EEG Features

### Class

In [13]:
class EEG_Features :
    
    def __init__(self, participant, clip):
        
        participant_data = data[participant-1]

        self.EEGF=participant_data["recordings"][clip]["EEG_features"]
        
        for i in range (0,len(self.EEGF)):
        
            if math.isnan(self.EEGF[i]) :
                
                self.EEGF[i]=0
    
    
    def get_x (self):
        
        return self.EEGF
    
    

def get_keys() :
    
    feature_list=[]
    
    for i in range (1, 89) :
        
        key="EEG_"+str(i)
        
        feature_list.append(key)        
        
    return feature_list

In [14]:
my=get_keys()
features=[*features[0], *my] # "features" is declared in ECG Data -> Answers -> 4

In [15]:
EEG_Features(1,1).get_x()

array([ 3.19802936e+02,  7.77842733e+01,  2.26021933e-01,  6.68486240e+00,
        9.74391006e-02,  1.07432854e-01, -9.16415405e-02,  5.35000000e-01,
        7.59525297e-01,  1.37564465e+00,  7.79512804e-01,  4.36164897e+02,
        1.20961906e+02, -2.30458070e-01,  1.88955345e+00,  2.14865709e-01,
        2.08619613e-01, -1.77698746e-01,  6.03125000e-01,  1.19925047e-01,
        1.28468001e+00,  1.39912555e-01,  6.24212680e+02,  1.52448509e+02,
       -3.71841108e-01,  2.84268586e+00,  1.47407870e-01,  1.41786384e-01,
        3.28766632e-02,  5.65625000e-01,  1.19925047e-01, -6.50878954e-02,
        1.39912555e-01,  2.48899126e+02,  9.49282376e+01,  9.05412382e-01,
        3.64877049e+00,  1.34291068e-01,  1.64272330e-01,  6.25640106e-02,
        5.75000000e-01,  1.19925047e-01, -4.86872624e-01,  1.39912555e-01,
        1.35783260e+02,  5.17711719e+01,  5.41444361e-01,  2.85951611e+00,
        1.36789507e-01,  1.88632105e-01, -2.54845428e-02,  5.91875000e-01,
        1.19925047e-01,  

## EMO 

### Class

In [16]:
class EMO:   
    
    def __init__(self, participant, clip):
        
        participant_data = data[participant-1]

        self.nEMO=participant_data["recordings"][clip]['EMO']
         
    #####################################################################################################    

    def EMO_stats(self):
        emoStats = []
        keys=['vdUL', 'vdLL', 'hdLLC', 'vdLLC', 'hdRLC', 'vdRLC', 'dRE', 'dLE', 'dRC', 'dLC', 'dRL', 'dLL']
        for i in range(1,13):
            
            aStat = []
            for j in range (0,len(self.nEMO)):
                if math.isnan(self.nEMO[j][i]):
                    continue
                else : 
                    aStat.append(self.nEMO[j][i]) 
            
            
            aStat_d=stats.describe(aStat) 
            aStat_d_mean, aStat_d_std = aStat_d[2], math.sqrt(aStat_d[3]) 
            aStat_d_PERQ1, aStat_d_PERQ3 = percentQ1Q3(aStat, aStat_d_mean, aStat_d_std)
            a_dict = {
                "EMO_"+keys[i-1]+"_mean":round(aStat_d_mean,3), 
                "EMO_"+keys[i-1]+"_std":round(aStat_d_std,3),
                "EMO_"+keys[i-1]+"_skw":round(aStat_d[4],3), 
                "EMO_"+keys[i-1]+"_krt":round(aStat_d[5],3),  
                "EMO_"+keys[i-1]+"_PERQ1":round(aStat_d_PERQ1,3),
                "EMO_"+keys[i-1]+"_PERQ3":round(aStat_d_PERQ3,3),
            }
            emoStats.append(a_dict)
                  
        emo= {**emoStats[0], **emoStats[1], **emoStats[2], **emoStats[3], **emoStats[4], **emoStats[5], **emoStats[6], **emoStats[7], **emoStats[8], **emoStats[9], **emoStats[10], **emoStats[11]}
        
        return emo
 

### Answer to questions

In [17]:
P1R1_EMO=EMO(1,1)

In [18]:
my=P1R1_EMO.EMO_stats().keys()
features=[*features, *my] 

In [19]:
P1R1_EMO.EMO_stats()

{'EMO_vdUL_mean': 0.077,
 'EMO_vdUL_std': 0.136,
 'EMO_vdUL_skw': 0.066,
 'EMO_vdUL_krt': 0.609,
 'EMO_vdUL_PERQ1': 0.138,
 'EMO_vdUL_PERQ3': 0.148,
 'EMO_vdLL_mean': -0.052,
 'EMO_vdLL_std': 0.037,
 'EMO_vdLL_skw': -0.327,
 'EMO_vdLL_krt': -0.075,
 'EMO_vdLL_PERQ1': 0.162,
 'EMO_vdLL_PERQ3': 0.143,
 'EMO_hdLLC_mean': -0.054,
 'EMO_hdLLC_std': 0.081,
 'EMO_hdLLC_skw': -0.276,
 'EMO_hdLLC_krt': 0.225,
 'EMO_hdLLC_PERQ1': 0.16,
 'EMO_hdLLC_PERQ3': 0.184,
 'EMO_vdLLC_mean': 0.262,
 'EMO_vdLLC_std': 0.186,
 'EMO_vdLLC_skw': 1.127,
 'EMO_vdLLC_krt': 1.551,
 'EMO_vdLLC_PERQ1': 0.075,
 'EMO_vdLLC_PERQ3': 0.141,
 'EMO_hdRLC_mean': 0.244,
 'EMO_hdRLC_std': 0.06,
 'EMO_hdRLC_skw': 0.344,
 'EMO_hdRLC_krt': 0.459,
 'EMO_hdRLC_PERQ1': 0.138,
 'EMO_hdRLC_PERQ3': 0.161,
 'EMO_vdRLC_mean': 0.343,
 'EMO_vdRLC_std': 0.237,
 'EMO_vdRLC_skw': 0.915,
 'EMO_vdRLC_krt': 0.687,
 'EMO_vdRLC_PERQ1': 0.171,
 'EMO_vdRLC_PERQ3': 0.128,
 'EMO_dRE_mean': 0.076,
 'EMO_dRE_std': 0.083,
 'EMO_dRE_skw': 2.433,
 'EMO_dRE

## EDA/GSM




### Helper Functions

In [20]:
def loadEDA(data, clip):
    """Load GSR from the data

    Parameters
    ----------
    data : dict
      dictionary of the participant.
    clip : int
      number of clip
      
    Return
    ------
    time : 1d-array
      The time series for the given clip when data is available.
    l1 : 1d-array
      The GSR series for the given clip when data is available.
    fs : float
      Sampling frequency.      
    """
        
    nEDA=data["recordings"][clip]['GSR']
    
    EDA, time = [], []
    
    for i in range (0,len(nEDA)-1):
        if math.isnan(nEDA[i][1]) or math.isnan(nEDA[i][0]) :
            continue
        else : 
            EDA.append(nEDA[i][1])
            time.append(nEDA[i][0])
        
    fs=data["FS_GSR"]
    
    return time, EDA, fs

####################################################################################################################

def butter_lowpass_filter(data, cutoff, fs, order):
    """Low pass filter used

    Parameters
    ----------
    data : 1d-array
      Input signal
    cutoff : float
      Cutoff frequency
    fs : float
      Sampling frequency
    order : int
      Order of the filter

    Return
    ------
    y : 1d-array
      Filtered signal
    """  
    nyq = 0.5 * fs
    
    normal_cutoff = cutoff / nyq
    
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    
    y = filtfilt(b, a, data)
    
    return y

####################################################################################################################

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """
    ______________________________________________________________________________
    Copyright (C) 2014-2015 Luca Citi, Alberto Greco
    ______________________________________________________________________________
     This method was first proposed in:
     A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
     "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
     IEEE Transactions on Biomedical Engineering, 2015
     DOI: 10.1109/TBME.2015.2474131
     ______________________________________________________________________________
    
    CVXEDA Convex optimization approach to electrodermal activity processing
    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see: http://cvxopt.org/userguide/coneprog.html#algorithm-parameters
    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return (np.array(a).ravel() for a in (r, p, t, l, d, e, obj))

### Class

In [21]:
class EDA:

    def __init__(self, participant, clip):
        
        participant_data = data[participant-1]
        
        self.time, self.EDA, self.fs = loadEDA(participant_data, clip)
        
        self.NormEDA=((self.EDA - statistics.mean(self.EDA) )/ statistics.stdev(self.EDA))

        self.gsr=butter_lowpass_filter(self.NormEDA, 5, self.fs, 1) #cutoff , order
        
        self.f, self.Pxx_den = signal.welch(self.gsr, self.fs, nperseg=self.fs*15, noverlap=self.fs*10)
        
        self.gsr50, self.time50, self.idx = last50Sec(self.time, self.gsr)
        
        
    #####################################################################################################           
    
    def plot_PSD(self):
        
        f, Pxx_den = signal.welch(self.gsr, self.fs, nperseg=self.fs*15, noverlap=self.fs*10)
        
        plt.plot(self.f, self.Pxx_den, 'b-', label='filtered')
        #plt.plot(f, Pxx_den, 'g-', label='non-filtered')
        plt.xlim([0, 1])
        plt.xlabel('Frequencies')
        plt.ylabel('PSD')
        plt.grid()
        plt.legend()
        plt.show()
    
    #####################################################################################################        
    
    def plot_filter(self):
        
        plt.plot(self.time, self.gsr, 'b-', label='filtered')
        #plt.plot(self.time, self.GSR50, 'g-', label='non-filtered')
        #plt.xlim([0,10])
        plt.xlabel('Time')
        plt.ylabel('Hz')
        plt.grid()
        plt.legend()
        plt.show()
        
        
    def plot_time(self):
                
        [r, p, t, l, d, e, obj] = cvxEDA(self.gsr, 1/self.fs)
                
        plt.plot(self.time, self.cond, 'b-', label='Normalized-Filtered EDA')
        #plt.plot(self.time, r, 'g-', label='Phasic SC Response')
        #plt.plot(self.time, p, 'y-', label='sparse SMNA driver of phasic component')
        #plt.plot(self.time, t, 'r-', label='Tonic SC Level')
        #plt.plot(self.time, self.y_spl[0], 'o-', label='1st Dev')
        
        plt.xlabel('Time')
        plt.ylabel('B')
        plt.grid()
        plt.legend()
        plt.show()
        
    #####################################################################################################        
    # resistance = self.r 
    
    def EDA_stats(self):
        
        self.cond50= np.array([1]*len(self.gsr50))/np.array(self.gsr50)
        
        # resistance 1st derivatives
        r_1dev=np.gradient(self.gsr50)/np.gradient(self.time50)
        r_1dev_neg = [item for item in r_1dev if item < 0]
        
        # conductance 1st derivatives
        y1=np.gradient(self.cond50)
        x1=np.gradient(self.time50)
        c_1dev= np.divide(y1, x1, out=np.zeros_like(y1), where=x1!=0)
        
        # conductance 2nd derivatives
        y2=np.gradient(y1)
        x2=np.gradient(x1)
        c_2dev= np.divide(y2, x2, out=np.zeros_like(y2), where=x2!=0)
        
        # 05
        exp, neg= 0,0
        for idx in range (0, len(r_1dev)-2):
            exp= exp+ (self.time50[idx+1]-self.time50[idx])
            if r_1dev[idx] < 0: 
                neg= neg+(self.time50[idx+1]-self.time50[idx])

        # 08
        rise = 0
        for idx in range (0, len(self.gsr50)-2):
            if (self.gsr50[idx+1] - self.gsr50[idx]) > 0: 
                rise = rise + (self.time50[idx+1]-self.time50[idx])
        
        
        # 09 - 4 very slow responses
        num=4
        band = [0,0.04]
        vs=[]
        subSTART=band[0]
        for i in range (0,num) : 
            subEND=subSTART+0.025
            vs.append(bandpower(np.array(self.gsr50), self.fs, [subSTART,subEND], nfft=len(self.gsr50)))
            subSTART=subSTART+0.005
        
        
        # 15 - 10 slow responses
        num=10
        band = [0,2.4]
        s= []
        subSTART=band[0]
        for i in range (0,num) : 
            subEND=subSTART+band[1]/num
            s.append(bandpower(np.array(self.gsr50), self.fs, [subSTART,subEND]))
            subSTART=subEND
        
        s_maxi, s_mini, s_std, s_area = bandpower(np.array(self.gsr50), self.fs, [0,2.4], nfft=len(self.gsr50) , additional= True)

        vs_maxi, vs_mini, vs_std, vs_area = bandpower(np.array(self.gsr50), self.fs, [0,0.04], nfft=len(self.gsr50) , additional= True)
        
        EDA_dict = {
            "EDA_vs1":vs[0],
            "EDA_vs2":vs[1],
            "EDA_vs3":vs[2],
            "EDA_vs4":vs[3],
            
                        
            "EDA_vs_max":vs_maxi,
            "EDA_vs_min":vs_mini,
            "EDA_vs_area":vs_area,
            "EDA_vs_std":vs_std,
            
            "EDA_s1":s[0],
            "EDA_s2":s[1],
            "EDA_s3":s[2],
            "EDA_s4":s[3],
            "EDA_s5":s[4],
            "EDA_s6":s[5],
            "EDA_s7":s[6],
            "EDA_s8":s[7],
            "EDA_s9":s[8],
            "EDA_s10":s[9],
            
            "EDA_s_max":s_maxi,
            "EDA_s_min":s_mini,
            "EDA_s_area":s_area,
            "EDA_s_std":s_std,
            
            "EDA_r_mean": statistics.mean(self.gsr50), #01
            "EDA_r_mean_1dev" : statistics.mean(r_1dev), #02
            "EDA_r_mean_1dev_abs" : statistics.mean(abs(r_1dev)), #03
            "EDA_r_mean_1dev_neg" : statistics.mean(r_1dev_neg), #04
            "EDA_r_mean_1dev_neg_perc" : neg/exp, #05
            "EDA_r_std":statistics.stdev(self.gsr50), #06
            "EDA_r_loc_min":len(argrelextrema(np.array(self.gsr50), np.less)[0]), #14
            "EDA_avg_rise" : rise/exp, #08
            
            "EDA_c_loc_min":len(argrelextrema(np.array(self.cond50), np.less)[0]), #07
            "EDA_c_std":statistics.stdev(self.cond50), #10
            "EDA_c_mean_1dev" : statistics.mean(c_1dev), #11
            "EDA_c_mean_1dev_abs" : statistics.mean(abs(c_1dev)), #12
            "EDA_c_mean_2dev_abs": statistics.mean(abs(c_2dev)), #13
        }
        
        return EDA_dict
    

### Answers to questions

In [22]:
P1R1_EDA=EDA(2,1)

In [23]:
my=P1R1_EDA.EDA_stats().keys()
features=[*features, *my] 

#### 1

In [24]:
import matplotlib.pyplot as plt
%matplotlib widget


P1R1_EDA.plot_PSD()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 2

In [25]:
import matplotlib.pyplot as plt
%matplotlib widget


P1R1_EDA.plot_filter()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 3

Galvanic skin resistance (GSR) 
- 08: average rising time of the GSR signal
- 09: power density estimates; 4 sub-bands in the [0-0.4] Hz band
- 15: log power density estimates; 10 sub-bands in the [0-2.4] Hz band
- 01: mean skin resistance
- 02: mean of first derivatives of skin resistance
- 03: mean of absolute values of first derivatives of skin resistance 
- 04: mean first derivative for negative values only
- 05: percentage of time with negative first derivative
- 06: standard deviation of skin resistance
- 14: number of local minima in the skin resistance signal

Conductance
- 07: number of local minima in the skin conductance signal
- 10: standard deviation of skin conductance
- 11: mean of first derivatives of skin conductance
- 12: mean of absolute values of first derivatives of skin conductance 
- 13: mean of absolute values of second derivatives of skin conductance 
- 07: number of local minima in the skin conductance signal

In [26]:
P1R1_EDA.EDA_stats()

{'EDA_vs1': 0.055,
 'EDA_vs2': 0.0,
 'EDA_vs3': 0.0,
 'EDA_vs4': 0.094,
 'EDA_vs_max': 5.526912387473278,
 'EDA_vs_min': 1.5890233091233288,
 'EDA_vs_area': 0.151,
 'EDA_vs_std': 1.9785259905152408,
 'EDA_s1': 1.065,
 'EDA_s2': 0.051,
 'EDA_s3': 0.005,
 'EDA_s4': 0.004,
 'EDA_s5': 0.003,
 'EDA_s6': 0.005,
 'EDA_s7': 0.001,
 'EDA_s8': 0.001,
 'EDA_s9': 0.0,
 'EDA_s10': 0.001,
 'EDA_s_max': 7.570020981210178,
 'EDA_s_min': 0.0011289742927930132,
 'EDA_s_area': 1.253,
 'EDA_s_std': 1.5602867909713318,
 'EDA_r_mean': -0.12001356505197441,
 'EDA_r_mean_1dev': 3.176472992819747e-07,
 'EDA_r_mean_1dev_abs': 0.0014193898879788422,
 'EDA_r_mean_1dev_neg': -0.0010786297316329198,
 'EDA_r_mean_1dev_neg_perc': 0.657705532979056,
 'EDA_r_std': 1.1029946901140928,
 'EDA_r_loc_min': 217,
 'EDA_avg_rise': 0.3466708346358237,
 'EDA_c_loc_min': 220,
 'EDA_c_std': 69.78487374061987,
 'EDA_c_mean_1dev': -1.4864031866124008e-05,
 'EDA_c_mean_1dev_abs': 0.11494806470336782,
 'EDA_c_mean_2dev_abs': 0.0}

#### 4

As for ECG, also in EDA PSD were used to capture additional features. Once again, the reason behind is this choice is academical. PSD of EDA were used in related work in emotional state recognition (see report). Several researches have considered statistical aspects (minimum, maximum, and variance signal magnitude area, skewness, kurtosis, harmonics summation) to predict emotional state with EDA signal. Thus, these have been integrated in the array features.

## Valence and Arousal

### Class

In [27]:
class VA :
    
    def __init__(self, partic, clip):
        
        participant_data = data[partic-1]
        
        self.arousal = participant_data["recordings"][clip]["arousal"]
        
        self.valence = participant_data["recordings"][clip]["valence"]
        
    def get_arousal(self) :

        return self.arousal
    
    def get_valence(self) :

        return self.valence

### Answers to questions

#### 1

In [146]:
valence, arousal = [], []

for i in range (1, 45) :
    
    for j in range(1, 37) : 
        
        try:
            
            PR_VA = VA(i, j)
            arousal.append(PR_VA.get_arousal())
            valence.append(PR_VA.get_valence())
            
        except KeyError:
            
              continue  
                
        except IndexError:
            
              continue  

        
print("Pearson coefficient : ", stats.pearsonr(np.array(valence), np.array(arousal)))

Pearson coefficient :  (-0.017677260717186068, 0.5004621723258766)


Valence and arousal are uncorrellated, which means that valence is a bad predictor of arousal and vice versa. The two variables are independent to one another. This finding is not surprising because these two indicators measure two different phenomena. 

#### 2

In [147]:
arousal_class, valence_class = [], []


for i in range (0, len(valence)) :
      
    
    if arousal[i]>3:
        
        
        arousal_class.append("high")
        
    else :
        
        arousal_class.append("low")   
        
        
    if valence[i]>0:

            
        valence_class.append("high")

    else :
        
        valence_class.append("low")

        
df=DataFrame(valence_class, columns=['valence_class'])

df['arousal_class']=arousal_class

df.groupby(['valence_class','arousal_class']).size().reset_index().rename(columns={0:'count'})

#df.describe()

,valence_class,arousal_class,count
0,high,high,483
1,high,low,188
2,low,high,473
3,low,low,311


This emotion classification system is inspired by dimensional theories of emotions. 

For the majority of the cases (65%), the clips watched by the participants provoked high level of arousal . Researchers probably selected video clips with extreme images because they wanted to provoke distinguishable and intense emotions. Positive and negative emotions are equally distributed . 

# Classification

## Extract and save features

In [148]:
final =[]

features=[*["participant", "clip"], *features, *["arousal", "valence"]]

final.append(features)

for i in range (1, 45) :
    
    for j in range(1, 37) : 
       
        try:
            
            PR_VA = VA(i, j)
            
            arousal = PR_VA.get_arousal()
            
            valence = PR_VA.get_valence()
            
            if arousal>3:
                arousal=1 #high

            else :
                arousal=0 #low   


            if valence>0:
                valence=1 #high
            else :
                valence=0 #low
            
            ecg = list(ECG(i, j).ECG_stats().values())
        
            eeg = EEG_Features(i,j).get_x()
            
            emo = list(EMO(i, j).EMO_stats().values())
            
            eda = list(EDA(i, j).EDA_stats().values())


            final.append([i, j,*ecg, *emo, *eda, *eeg, arousal, valence])

        except KeyError:
            
              continue  
                
        except IndexError:
            
              continue  

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [149]:
import pickle

with open('featurelist', 'wb') as f:
     pickle.dump(final, f) 

## Aggregation data

In [34]:
with open('featurelist', 'rb') as f:
      final = pickle.load(f)

In [35]:
def replaceNANwithMean(dataframe) :
    """Replace NaN with their Mean and zero infinitive values

    Parameters
    ----------
    dataframe : dataframe
      dataframe of features and labels.
      
    Return
    ------
    dataframe : dataframe
      dataframe with only valid data.  
    """
            
    dataframe.replace([np.inf, -np.inf], 0, inplace=True)
    
    keys = list(ECG(1,1).ECG_stats().keys())
    
    for key in keys : 
        
        mean=dataframe[key].loc[dataframe[key] != "nan"].mean()
        
        dataframe[key].loc[(dataframe[key] == "nan")] = mean
    
    return dataframe

def leaveParticipant (dataframe, num) :
    
    """LOPO validation scheme

    Parameters
    ----------
    dataframe : dataframe
      dataframe of features and labels.
    num : int
      participant to predict
      
    Return
    ------
    X_train, X_test, y_train, y_test : 4 arrays (2x249-d arrays, 2x2-d arrays )
      training and validation arrays 
    """
    
    
    leaveParticipant = dataframe.loc[dataframe["participant"] != num]

    X_train = leaveParticipant.drop("arousal", axis=1).drop("valence", axis=1).to_numpy()
    y_train = leaveParticipant[["arousal","valence" ]].to_numpy()

    Participant = dataframe.loc[dataframe["participant"] == num]
    Participant = Participant.dropna().reset_index(drop=True)

    X_test = Participant.drop("arousal", axis=1).drop("valence", axis=1).to_numpy()
    y_test = Participant[["arousal","valence" ]].to_numpy()
    
    return X_train, X_test, y_train, y_test
    

def leaveClip (dataframe, num) :
    
    """LOCO validation scheme

    Parameters
    ----------
    dataframe : dataframe
      dataframe of features and labels.
    num : int
      clip to predict
      
    Return
    ------
    X_train, X_test, y_train, y_test : 4 arrays (2x249-d arrays, 2x2-d arrays )
      training and validation arrays 
    """
    
    #train
    leaveClip = dataframe.loc[dataframe["clip"] != num]
    X_train = leaveClip.drop("arousal", axis=1).drop("valence", axis=1).drop("clip", axis=1).drop("participant", axis=1).to_numpy()
    y_train = leaveClip[["arousal","valence" ]].to_numpy()

    #test
    Participant = dataframe.loc[dataframe["clip"] == num]
    X_test = Participant.drop("arousal", axis=1).drop("valence", axis=1).drop("clip", axis=1).drop("participant", axis=1).to_numpy()
    y_test = Participant[["arousal","valence" ]].to_numpy()
    
    return X_train, X_test, y_train, y_test

In [36]:
dataframe = pd.DataFrame(final[1:], columns=final[0])
dataframe=replaceNANwithMean(dataframe)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Predictors & stats

In [310]:
class classification :
    
    def __init__(self, dataframe, num, method) :
        
        self.num=num
        
        if (method == "participant") :
            
            self.X_train, self.X_test, self.y_train, self.y_test = leaveParticipant(dataframe, num)

        else :
            self.X_train, self.X_test, self.y_train, self.y_test = leaveClip(dataframe, num)

            
    def RandomForestClassifier(self) :

        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

        ### Random Forest Classifier

        clf=RandomForestClassifier(n_estimators=300)

        clf.fit(self.X_train,self.y_train)

        y_pred=clf.predict(self.X_test)

        acc=round(accuracy_score(self.y_test, y_pred),2)
        f1=round(f1_score(self.y_test, y_pred, average='micro'),2)
        pre=round(precision_score(self.y_test, y_pred, average='micro'),2)
        rec=round(recall_score(self.y_test, y_pred, average='micro'),2)
        
        arousal_cm=confusion_matrix([item[0] for item in self.y_test], [item[0] for item in y_pred], labels=[0,1])
        
        valence_cm=confusion_matrix([item[1] for item in self.y_test], [item[1] for item in y_pred], labels=[0,1])
        
        return [self.num, acc, f1, pre, rec, arousal_cm, valence_cm]
    
    
    def ZeroRuleClassifier(self) :

        from sklearn.dummy import DummyClassifier
        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

        ### Zero Rule Classifier

        dummy_clf = DummyClassifier(strategy="most_frequent")

        dummy_clf.fit(self.X_train,self.y_train)

        y_pred=dummy_clf.predict(self.X_test)

        acc=round(accuracy_score(self.y_test, y_pred),2)
        f1=round(f1_score(self.y_test, y_pred, average="micro"),2)
        pre=round(precision_score(self.y_test, y_pred, average="micro"),2)
        rec=round(recall_score(self.y_test, y_pred, average="micro"),2)
        
        arousal_cm=confusion_matrix([item[0] for item in self.y_test], [item[0] for item in y_pred], labels=[0,1])
        
        valence_cm=confusion_matrix([item[1] for item in self.y_test], [item[1] for item in y_pred], labels=[0,1])

        return [self.num, acc, f1, pre, rec, arousal_cm, valence_cm]
    

## Measurements

In [311]:
rfc_part, zrc_part, rfc_clip, zrc_clip = [], [], [], []

for i in range (1, 45) : 
    
    test=classification(dataframe, i, "participant")
    
    rfc_part.append(test.RandomForestClassifier())
    
    zrc_part.append(test.ZeroRuleClassifier())
    
for i in range (1, 37) : 
    
    test=classification(dataframe, i, "clip")
    
    rfc_clip.append(test.RandomForestClassifier())
    
    zrc_clip.append(test.ZeroRuleClassifier())

In [312]:
import pickle

with open('rfc_part', 'wb') as f:
     pickle.dump(rfc_part, f) 
            
with open('zrc_part', 'wb') as f:
     pickle.dump(zrc_part, f) 
        
with open('rfc_clip', 'wb') as f:
     pickle.dump(rfc_clip, f) 
        
with open('zrc_clip', 'wb') as f:
     pickle.dump(zrc_clip, f) 

## Answer to questions

In [37]:
import pickle

with open('zrc_clip', 'rb') as f:
      zrc_clip = pickle.load(f)
        
with open('rfc_part', 'rb') as f:
      rfc_part = pickle.load(f)
        
with open('zrc_part', 'rb') as f:
      zrc_part = pickle.load(f)
        
with open('rfc_clip', 'rb') as f:
      rfc_clip = pickle.load(f)

### 1

In [38]:
from tabulate import tabulate

headers=['Participant','Accuracy', 'F1', 'Precision', 'Recall', 'Confusion Matrix Arousal', 'Confusion Matrix Valence']

print(tabulate(rfc_part, headers= headers))

  Participant    Accuracy    F1    Precision    Recall  Confusion Matrix Arousal    Confusion Matrix Valence
-------------  ----------  ----  -----------  --------  --------------------------  --------------------------
            1        0.64  0.79         0.67      0.97  [[ 0 11]                    [[17  7]
                                                         [ 0 25]]                    [ 1 11]]
            2        0.75  0.9          0.85      0.96  [[ 0  7]                    [[16  1]
                                                         [ 0 29]]                    [ 2 17]]
            3        0.8   0.93         0.89      0.98  [[ 0  6]                    [[16  0]
                                                         [ 0 29]]                    [ 1 18]]
            4        0.44  0.73         0.59      0.94  [[ 0 18]                    [[16  4]
                                                         [ 0 18]]                    [ 2 14]]
            5        0.54  0.75 

In [39]:
from tabulate import tabulate

headers=['Clip','Accuracy', 'F1', 'Precision', 'Recall', 'Confusion Matrix Arousal', 'Confusion Matrix Valence']

print(tabulate(rfc_clip, headers= headers))

  Clip    Accuracy    F1    Precision    Recall  Confusion Matrix Arousal    Confusion Matrix Valence
------  ----------  ----  -----------  --------  --------------------------  --------------------------
     1        0.15  0.59         0.61      0.58  [[ 0 15]                    [[ 7  4]
                                                  [ 0 24]]                    [22  6]]
     2        0.28  0.69         0.75      0.64  [[ 0 13]                    [[ 8  0]
                                                  [ 1 26]]                    [20 12]]
     3        0.26  0.68         0.68      0.68  [[ 0 14]                    [[10  2]
                                                  [ 0 24]]                    [16 10]]
     4        0.44  0.79         0.85      0.73  [[ 0  6]                    [[10  2]
                                                  [ 0 35]]                    [17 12]]
     5        0.44  0.82         0.95      0.73  [[ 0  2]                    [[ 2  1]
                

### 2

In [40]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

arousal=list(map(list, zip(*rfc_part)))[-2]
valence=list(map(list, zip(*rfc_part)))[-1]

sum_cf_ar = np.zeros((2, 2))
sum_cf_va = np.zeros((2, 2))

for i in range (0,len(arousal)):
    sum_cf_ar=sum_cf_ar+arousal[i]
    sum_cf_va=sum_cf_va+valence[i]

disp = ConfusionMatrixDisplay(confusion_matrix=sum_cf_ar, display_labels=[0,1])
disp.plot(values_format=".1f", cmap=plt.cm.Blues,) 

disp = ConfusionMatrixDisplay(confusion_matrix=sum_cf_va, display_labels=[0,1])
disp.plot(values_format=".1f", cmap=plt.cm.Blues,) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
arousal=list(map(list, zip(*rfc_clip)))[-2]
valence=list(map(list, zip(*rfc_clip)))[-1]

sum_cf_ar = np.zeros((2, 2))
sum_cf_va = np.zeros((2, 2))

for i in range (0,len(arousal)):
    sum_cf_ar=sum_cf_ar+arousal[i]
    sum_cf_va=sum_cf_va+valence[i]


disp = ConfusionMatrixDisplay(confusion_matrix=sum_cf_ar, display_labels=[0,1])
disp.plot(values_format=".1f", cmap=plt.cm.Blues,) 

disp = ConfusionMatrixDisplay(confusion_matrix=sum_cf_va, display_labels=[0,1])
disp.plot(values_format=".1f", cmap=plt.cm.Blues,) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3

In [42]:
corr_matrix = dataframe.corr()
pd.set_option('display.max_rows', 10)
top10Aro=corr_matrix["arousal"].abs().sort_values(ascending=False)[1:11]
top10Val=corr_matrix["valence"].abs().sort_values(ascending=False)[1:11]

print("10 features for arousal prediction : ", top10Aro)
print("10 features for valence prediction : ", top10Val)

10 features for arousal prediction :  valence           0.120789
EMO_vdLL_PERQ1    0.102161
EMO_hdLLC_std     0.101611
clip              0.076309
EDA_vs_area       0.065592
EMO_dLE_mean      0.064913
EEG_15            0.059638
EEG_8             0.058480
EEG_12            0.056818
EEG_13            0.056427
Name: arousal, dtype: float64
10 features for valence prediction :  clip              0.602941
arousal           0.120789
EEG_19            0.118785
EMO_vdLL_PERQ3    0.099383
EEG_32            0.089235
EEG_62            0.071926
EEG_76            0.071193
EEG_80            0.070698
EMO_vdLL_mean     0.070512
EEG_77            0.070239
Name: valence, dtype: float64


### 4

In [43]:
from tabulate import tabulate

headers=['Classifier', 'Accuracy', 'F1', 'Precision', 'Recall']

part, list_rfc, list_zrc = [], ['Random Forest'], ['Zero Rule']

list_rfc.append(round(statistics.median([item[1] for item in rfc_part]),2))
list_rfc.append(round(statistics.median([item[2] for item in rfc_part]),2))
list_rfc.append(round(statistics.median([item[3] for item in rfc_part]),2))
list_rfc.append(round(statistics.median([item[4] for item in rfc_part]),2))

list_zrc.append(round(statistics.median([item[1] for item in zrc_part]),2))
list_zrc.append(round(statistics.median([item[2] for item in zrc_part]),2))
list_zrc.append(round(statistics.median([item[3] for item in zrc_part]),2))
list_zrc.append(round(statistics.median([item[4] for item in zrc_part]),2))

part.append(list_rfc)
part.append(list_zrc)  

print(tabulate(part, headers= headers))

Classifier       Accuracy    F1    Precision    Recall
-------------  ----------  ----  -----------  --------
Random Forest        0.57   0.8         0.7       0.95
Zero Rule            0.33   0.6         0.66      0.59


In [44]:
from tabulate import tabulate

headers=['Classifier', 'Accuracy', 'F1', 'Precision', 'Recall']

part, list_rfc, list_zrc = [], ['Random Forest'], ['Zero Rule']

list_rfc.append(round(statistics.median([item[1] for item in rfc_clip]),2))
list_rfc.append(round(statistics.median([item[2] for item in rfc_clip]),2))
list_rfc.append(round(statistics.median([item[3] for item in rfc_clip]),2))
list_rfc.append(round(statistics.median([item[4] for item in rfc_clip]),2))

list_zrc.append(round(statistics.median([item[1] for item in zrc_clip]),2))
list_zrc.append(round(statistics.median([item[2] for item in zrc_clip]),2))
list_zrc.append(round(statistics.median([item[3] for item in zrc_clip]),2))
list_zrc.append(round(statistics.median([item[4] for item in zrc_clip]),2))

part.append(list_rfc)
part.append(list_zrc)  

print(tabulate(part, headers= headers))

Classifier       Accuracy    F1    Precision    Recall
-------------  ----------  ----  -----------  --------
Random Forest        0.36  0.68         0.61      0.73
Zero Rule            0.21  0.61         0.67      0.54


# Code - References 

[1] Mojtaba Khomami Abadi, Ramanathan Subramanian, Seyed Mostafa Kia, Paolo Avesani, Ioannis Patras, and Nicu Sebe. Decaf: Meg-based multimodal database for decoding affective physiolog- ical responses. IEEE Transactions on Affective Computing, 6(3):209–222, 2015.

[2] http://ems12lead.com/2014/03/10/understanding-ecg-filtering/

[3] http://www.jscholaronline.org/articles/JBER/Signal-Processing.pdf
